Ricerca e controllo per possibili outlier

Import delle librerie necessarie


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Inizio della parte di ricerca di possibili outlier.


#Inizio delle ricerca sui prezzi delle auto


In [ ]:
df_clean = pd.read_csv(r"..\data\processed\database_cleaned_2.csv")
df_clean.max()['Price']
df_clean.min()['Price']
df_clean['Price'].mean()
df_clean['Price'].median()
print("Media:", df_clean['Price'].mean())
print("Mediana:", df_clean['Price'].median())
print("Massimo:",df_clean.max()['Price'])
print("Minimo:",df_clean.min()['Price'])
top_10_prezzi = df_clean.nlargest(10, 'Price')
print(top_10_prezzi[['Price']])
bottom_10_prezzi = df_clean.nsmallest(10, 'Price')
print(bottom_10_prezzi[['Price']])


#Dai risultati abbiamo 6 valori che risutano anomali. Si tratta dei 6 prezzi più bassi.


#Ricerca più approfondita riguradante gli outlier, con prezzi di controllo, per vedere quanto impattano sul dataset.


In [ ]:
outlier_alti = df_clean[df_clean['Price'] > 80000]
outlier_bassi = df_clean[df_clean['Price'] < 9000]
num_outlier_alti = len(outlier_alti)
num_outlier_bassi = len(outlier_bassi)
print(f"\nNumero di outlier con prezzo > 80000: {num_outlier_alti}")
print(f"Numero di outlier con prezzo < 9000: {num_outlier_bassi}")
df_no_outliers = df_clean[(df_clean['Price'] <= 80000) & (df_clean['Price'] >= 9000)]
print(f"\nDimensione del dataset originale: {len(df_clean)}")
print(f"Dimensione del dataset senza outlier: {len(df_no_outliers)}")

#Il prezzo di controllo della fascia alta mostra un gruppo compatto e in sincronia con il resto del dataset.

#Il prezzo di controlo della fascia bassa invece mostra nuovamente i 6 prezzi più bassi che sono discostati dal resto del dataset.

#Inzio della ricerca su "Annual Income"

In [ ]:
df_clean.max()['Annual Income']
df_clean.min()['Annual Income']
df_clean['Annual Income'].mean()
df_clean['Annual Income'].median()
print("Media:", df_clean['Annual Income'].mean())
print("Mediana:", df_clean['Annual Income'].median())
print("Massimo:",df_clean.max()['Annual Income'])
print("Minimo:",df_clean.min()['Annual Income'])
top_10_income = df_clean.nlargest(10, 'Annual Income')
print(top_10_income[['Annual Income']])
bottom_10_income = df_clean.nsmallest(10, 'Annual Income')
print(bottom_10_income[['Annual Income']])



#La ricerca preliminare evidenzia valori relativamente alti. Ci sono dei valori che si potrrebbero considerare anomali.

#Ricerca più approfondita riguradante gli outlier, con gruppi di controllo, per vedere quanto impattano sul dataset.

In [ ]:
outlier_salari_alti = df_clean[df_clean['Annual Income'] > 7000000]
outlier_salari_bassi = df_clean[df_clean['Annual Income'] < 13500]
num_outlier_salari_alti = len(outlier_salari_alti)
num_outlier_salari_bassi = len(outlier_salari_bassi)
print(f"\nNumero di outlier con salari > 7000000: {num_outlier_salari_alti}")
print(f"Numero di outlier con salari < 13500: {num_outlier_salari_bassi}")
df_no_outliers = df_clean[(df_clean['Annual Income'] <= 7000000) & (df_clean['Annual Income'] >= 13500)]
print(f"\nDimensione del dataset originale: {len(df_clean)}")
print(f"Dimensione del dataset senza outlier: {len(df_no_outliers)}")

#Il prezzo di controllo della fascia bassa mostra un gruppo compatto e in sincronia con il resto del dataset.

#Il prezzo di controlo della fascia alta invece mostra mostra 3 valori più alti rispetto al resto dei data, ma non abbastanza da risultare troppo anomali.

#Ricerca con utilizzo di "group by" per vedere come i dati sono correlati.Prima per "Annual Income" e poi per "Price".

In [ ]:
top_10_redditi = df_clean.sort_values(by='Annual Income', ascending=False).head(10)
print("\nLe 10 auto con il reddito annuale più alto:")
print(top_10_redditi[['Company', 'Model', 'Annual Income', 'Price']])
bottom_10_redditi = df_clean.sort_values(by='Annual Income', ascending=False).tail(10)
print("\nLe 10 auto con il reddito annuale più basso:")
print(bottom_10_redditi[['Company', 'Model', 'Annual Income', 'Price']])


In [ ]:
top_10_auto = df_clean.sort_values(by='Price', ascending=False).head(10)
print("\nLe 10 auto con il prezzo più alto:")
print(top_10_auto[['Company', 'Model', 'Annual Income', 'Price']])
bottom_10_auto = df_clean.sort_values(by='Price', ascending=False).tail(10)
print("\nLe 10 auto con il prezzo più basso:")
print(bottom_10_auto[['Company', 'Model', 'Annual Income', 'Price']])

#A seguito dei dati ottenuti dalle esplorazioni e al confronto con gli altri membri del team, si è deciso di eliminare gli outlier evidenziati dalla ricerca sui prezzi delle auto che ha mostarto 6 dati anomali.